In [4]:
import cv2
import numpy as np
from PIL import Image

In [5]:
 with open("dataset\Videos\data_test1.rgb", "rb") as f:
    v = np.fromfile(f, np.dtype('B'))

In [6]:
#setting the number of pixels in frame
pxct = 129600
#setting the number of frames
frct = (int)(v.size/(3*pxct))

In [7]:
final_video = []
for frame_number in range(frct):
        start = (frame_number) * pxct * 3
        curr_frame = v[start:start + 3 * pxct]
        final_frame = np.reshape(curr_frame, ( 480,270, 3), order = 'F')
        final_frame = np.rot90(final_frame,3)
        final_frame = np.flip(final_frame,1)
        final_video.append(final_frame)

In [8]:
#create the video
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))

for i in range(frct):
    out.write(final_video[i])

out.release()

In [9]:
from skimage.metrics import structural_similarity

In [26]:
new_shot = []
for frame_number in range(frct - 1):
    # Convert images to grayscale
    hist1 = cv2.calcHist([final_video[frame_number]],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([final_video[frame_number + 1]],[0],None,[256],[0,256])
    score=cv2.compareHist(hist1,hist2,cv2.HISTCMP_BHATTACHARYYA)
    if score > 0.2:
        new_shot.append(frame_number)
        data = Image.fromarray(final_video[frame_number]).convert('RGB')
        data.save('images/{}.png'.format(frame_number))
        data = Image.fromarray(final_video[frame_number+1]).convert('RGB')
        data.save('images/{}.png'.format(frame_number+1))

In [11]:
i1 = cv2.cvtColor(final_video[frame_number], cv2.COLOR_BGR2GRAY)
i2 = cv2.cvtColor(final_video[frame_number - 1], cv2.COLOR_BGR2GRAY)
(score, diff) = structural_similarity(i1, i2, full=True)
print("Structural Similarity Index: {}".format(score))

Structural Similarity Index: 0.16868839552965767


In [21]:
hist1 = cv2.calcHist([final_video[0]],[0],None,[256],[0,256])
hist2 = cv2.calcHist([final_video[5]],[0],None,[256],[0,256])
a=cv2.compareHist(hist1,hist2,cv2.HISTCMP_BHATTACHARYYA)

In [22]:
a

0.08054781646879208

In [15]:
new_shot

[605,
 1945,
 2012,
 2023,
 2024,
 2025,
 2027,
 2034,
 2035,
 2040,
 2041,
 2044,
 2045,
 2046,
 2047,
 2141,
 2189,
 2190,
 2223,
 2225,
 2226,
 2227,
 2228,
 2231,
 2232]